## Creating Dataset for Heatmap 

Resources used: 
- https://stackoverflow.com/questions/46161110/expand-pandas-date-range
- https://stackoverflow.com/questions/22650833/pandas-groupby-cumulative-sum

In [627]:
import pandas as pd 

In [628]:
df_data = pd.read_csv("2021VAERSDATA.csv", engine='python')

In [629]:
df_data.shape

(34121, 35)

In [630]:
states = pd.read_csv('state.csv')

In [631]:
df = df_data.loc[:, ('RECVDATE','STATE', 'DIED')]
df.rename(columns={"STATE": "state","RECVDATE": "date", "DIED": "died" }, inplace=True)

In [632]:
df

,date,state,died
0,01/01/2021,TX,NaN
1,01/01/2021,CA,NaN
2,01/01/2021,WA,NaN
3,01/01/2021,WA,NaN
4,01/01/2021,TX,NaN
...,...,...,...
34116,03/19/2021,FL,NaN
34117,03/19/2021,PA,NaN
34118,03/19/2021,MS,Y
34119,03/19/2021,AZ,Y


In [633]:
aa = df['state'].value_counts()
print( str(aa.shape[0]) + ' is the amount of states in the dataset')

60 is the amount of states in the dataset


In [634]:
df['died'].value_counts()

Y    1957
Name: died, dtype: int64

In [635]:
df['died'] = (df.loc[:,'died'] == 'Y').astype(int)

In [636]:
df

,date,state,died
0,01/01/2021,TX,0
1,01/01/2021,CA,0
2,01/01/2021,WA,0
3,01/01/2021,WA,0
4,01/01/2021,TX,0
...,...,...,...
34116,03/19/2021,FL,0
34117,03/19/2021,PA,0
34118,03/19/2021,MS,1
34119,03/19/2021,AZ,1


In [637]:
import numpy as np
counts  = np.ones(len(df))
df['counts'] = counts

In [638]:
df

,date,state,died,counts
0,01/01/2021,TX,0,1.0
1,01/01/2021,CA,0,1.0
2,01/01/2021,WA,0,1.0
3,01/01/2021,WA,0,1.0
4,01/01/2021,TX,0,1.0
...,...,...,...,...
34116,03/19/2021,FL,0,1.0
34117,03/19/2021,PA,0,1.0
34118,03/19/2021,MS,1,1.0
34119,03/19/2021,AZ,1,1.0


In [639]:
df.groupby(['state', 'date']).sum()

died  counts
state date                    
AK    01/01/2021     0     2.0
      01/02/2021     0     2.0
      01/04/2021     0     3.0
      01/05/2021     0    13.0
      01/06/2021     0    20.0
...                ...     ...
WY    03/08/2021     2     2.0
      03/10/2021     0     1.0
      03/11/2021     0     1.0
      03/12/2021     1     2.0
XB    01/04/2021     0     1.0

[2978 rows x 2 columns]

In [640]:
df = df.groupby(['state', 'date']).sum() \
  .groupby(level=0).cumsum().reset_index()

In [641]:
df

,state,date,died,counts
0,AK,01/01/2021,0,2.0
1,AK,01/02/2021,0,4.0
2,AK,01/04/2021,0,7.0
3,AK,01/05/2021,0,20.0
4,AK,01/06/2021,0,40.0
...,...,...,...,...
2973,WY,03/08/2021,8,49.0
2974,WY,03/10/2021,8,50.0
2975,WY,03/11/2021,8,51.0
2976,WY,03/12/2021,9,53.0


In [642]:
df.date = pd.to_datetime(df.date)
df = df.set_index('date').groupby('state').resample('1D')[('counts','died')].ffill().reset_index()

In [643]:
df

,state,date,counts,died
0,AK,2021-01-01,2.0,0
1,AK,2021-01-02,4.0,0
2,AK,2021-01-03,4.0,0
3,AK,2021-01-04,7.0,0
4,AK,2021-01-05,20.0,0
...,...,...,...,...
4129,WY,2021-03-09,49.0,8
4130,WY,2021-03-10,50.0,8
4131,WY,2021-03-11,51.0,8
4132,WY,2021-03-12,53.0,9


In [644]:
state_n = df['state'].value_counts()
state_n.index[0]


'AZ'

In [645]:
len_p = 78
dr = pd.date_range("2021-01-01", periods=len_p, freq="D")
names = ['AZ'] * len_p
d = {'state':names, 'date':dr}
df_new = pd.DataFrame(d)

In [646]:
state_n = df['state'].value_counts()
state_n.index[1:len(state_n)]

for i in state_n.index[1:len(state_n)]: 
    names = [i] * len_p
    dates = dr
    d = {'state':names, 'date':dates}
    df_temp = pd.DataFrame(d)
    df_new = df_new.append(df_temp)
    
df_new

,state,date
0,AZ,2021-01-01
1,AZ,2021-01-02
2,AZ,2021-01-03
3,AZ,2021-01-04
4,AZ,2021-01-05
...,...,...
73,FM,2021-03-15
74,FM,2021-03-16
75,FM,2021-03-17
76,FM,2021-03-18


In [647]:
#for mulitple selected states
temp = df.set_index("date")
temp = temp[temp['state'].isin(['AK','CA'])].groupby('date').sum()

In [648]:
df_new.dtypes

state            object
date     datetime64[ns]
dtype: object

In [649]:
df_state = (df.merge(states, left_on='state', right_on='state')
          .reindex(columns=['date', 'state', 'latitude', 'longitude','counts', 'died']))


new_df = pd.merge(df_new, df_state,  how='left', left_on=['date','state'], right_on = ['date','state'])

In [650]:
df_state.shape

(3950, 6)

In [658]:
new_df.groupby(['state','date']).count()

latitude  longitude  counts  died
state date                                         
AK    2021-01-01         1          1       1     1
      2021-01-02         1          1       1     1
      2021-01-03         1          1       1     1
      2021-01-04         1          1       1     1
      2021-01-05         1          1       1     1
...                    ...        ...     ...   ...
XB    2021-03-15         0          0       0     0
      2021-03-16         0          0       0     0
      2021-03-17         0          0       0     0
      2021-03-18         0          0       0     0
      2021-03-19         0          0       0     0

[4680 rows x 4 columns]

In [623]:
#new_df.dropna(subset = ["latitude"], inplace=True)

In [661]:
new_df = new_df[new_df['state'].isin(df_state['state'].value_counts().index)]

In [670]:
new_df.latitude

0       34.048928
1       34.048928
2       34.048928
3       34.048928
4       34.048928
          ...    
4051          NaN
4052          NaN
4053          NaN
4054          NaN
4055          NaN
Name: latitude, Length: 4056, dtype: float64

In [682]:
new_df.loc[:,'latitude']

0       34.048928
1       34.048928
2       34.048928
3       34.048928
4       34.048928
          ...    
4051    43.969515
4052    43.969515
4053    43.969515
4054    43.969515
4055    43.969515
Name: latitude, Length: 4056, dtype: float64

In [684]:
new_df.groupby(['state'])['latitude'].ffill()

0       34.048928
1       34.048928
2       34.048928
3       34.048928
4       34.048928
          ...    
4051    43.969515
4052    43.969515
4053    43.969515
4054    43.969515
4055    43.969515
Name: latitude, Length: 4056, dtype: float64

In [686]:
new_df.groupby('state').count()

fill_names = ['latitude', 'longitude', 'counts', 'died']

for i in fill_names: 
    new_df[i] = new_df.groupby(['state'])[i].ffill()
    
new_df.groupby('state').count()

<ipython-input-686-5a2a04cfa040>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,latitude,longitude,counts,died
state,,,,,
AK,78,78,78,78,78
AL,78,78,78,78,78
AR,78,78,78,78,78
AZ,78,78,78,78,78
CA,78,78,78,78,78
CO,78,78,78,78,78
CT,78,78,78,78,78
DC,78,78,78,78,78
DE,78,78,78,78,78


In [690]:
new_df

,state,date,latitude,longitude,counts,died
0,AZ,2021-01-01,34.048928,-111.093731,14.0,0.0
1,AZ,2021-01-02,34.048928,-111.093731,15.0,0.0
2,AZ,2021-01-03,34.048928,-111.093731,21.0,0.0
3,AZ,2021-01-04,34.048928,-111.093731,58.0,0.0
4,AZ,2021-01-05,34.048928,-111.093731,97.0,0.0
...,...,...,...,...,...,...
4051,SD,2021-03-15,43.969515,-99.901813,116.0,12.0
4052,SD,2021-03-16,43.969515,-99.901813,116.0,12.0
4053,SD,2021-03-17,43.969515,-99.901813,116.0,12.0
4054,SD,2021-03-18,43.969515,-99.901813,116.0,12.0


In [625]:
print(str(min(df_state['date'])) , str(max(df_state['date'])))

2021-01-01 00:00:00 2021-03-19 00:00:00


In [626]:
pd.to_datetime(max(df['date'])) - pd.to_datetime(min(df['date']))

Timedelta('77 days 00:00:00')

In [691]:
#new_df.to_csv('/Users/vishnu/Documents/ECS 171/Final Project/heatmap_ecs171/df_state.csv')

In [597]:
new_df.to_csv('df_state.csv')